In [1]:
!pip install keras
!pip install sklearn
!pip install matplotlib
!pip install -U -q PyDrive

  Running setup.py bdist_wheel for sklearn ... - done
  Stored in directory: /content/.cache/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn


In [2]:
from keras import backend as K
K.clear_session()

Using TensorFlow backend.


In [0]:
from keras import backend as K
cfg = K.tf.ConfigProto()
cfg.gpu_options.allow_growth = True
K.set_session(K.tf.Session(config=cfg))

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
file_import = drive.CreateFile({'id':'1p1bsltfTcIrZ_kfE6kwGTPzcdXorHbb2'})
file_import.GetContentFile('colab_setup.py') 
from colab_setup import setup

setup(drive)

In [6]:
!ls

colab_setup.py			 roc_auc_callback.py	X_train.npy
custom_fast_text_embeddings.npy  sample_submission.csv	X_val.npy
datalab				 train_model.py		y_test.npy
fast_text_embeddings.npy	 X_submission.npy	y_train_full.npy
plot_history.py			 X_test.npy		y_train.npy
__pycache__			 X_train_full.npy	y_val.npy


In [0]:
import numpy as np
import os

from plot_history import plot_history
from roc_auc_callback import RocAucCallback
from train_model import train_with_cv, train_with_submitting, evaluate_on_test

In [0]:
from keras.models import Model
from keras.layers import *
from keras.activations import *

In [0]:
def get_lstm_last_step(lstm_sequences):
    return Lambda(lambda sequence: sequence[:, -1, :])(lstm_sequences)

def classification_block(input_layer, dropout_rate=0.2):
    model = Dense(256, activation='linear',
                  kernel_initializer='he_normal')(input_layer)
    model = BatchNormalization()(model)
    model = ELU()(model)
    model = Dropout(dropout_rate)(model)

    model = Dense(256, activation='linear',
                  kernel_initializer='he_normal')(model)
    model = BatchNormalization()(model)
    model = ELU()(model)
    model = Dropout(dropout_rate)(model)

    return Dense(6, activation='sigmoid', kernel_initializer='glorot_normal')(model)

In [0]:
max_words = 30000
max_words_length = 400
embedding_dimension = 300

lstm_options = {
    'units': 64,
    'dropout': 0.2,
    'recurrent_dropout': 0.2,
    'return_sequences': True
}

In [11]:
input_layer = Input(shape=(max_words_length,))

pretrained_embedding_layer = Embedding(
    max_words,
    output_dim=embedding_dimension,
    input_length=max_words_length,
    weights=[np.load('fast_text_embeddings.npy')],
    trainable=False
)(input_layer)
pretrained_embedding_layer = SpatialDropout1D(0.2)(pretrained_embedding_layer)

custom_embedding_layer = Embedding(
    max_words,
    output_dim=embedding_dimension,
    input_length=max_words_length,
    weights=[np.load('custom_fast_text_embeddings.npy')],
    trainable=False
)(input_layer)
custom_embedding_layer = SpatialDropout1D(0.2)(custom_embedding_layer)

averaged_embedding_layer = Average()(
    [pretrained_embedding_layer, custom_embedding_layer])
averaged_embedding_layer = SpatialDropout1D(0.2)(averaged_embedding_layer)

lstm = Bidirectional(
  LSTM(
    units=lstm_options['units'],
    dropout=lstm_options['dropout'],
    recurrent_dropout=lstm_options['recurrent_dropout'],
    return_sequences=lstm_options['return_sequences']
  )
)(averaged_embedding_layer)

pooled_features = concatenate([
    GlobalMaxPooling1D()(lstm),
    GlobalAveragePooling1D()(lstm),
    get_lstm_last_step(lstm)
])

pooled_features = Dropout(0.2)(pooled_features)

output_layer = classification_block(pooled_features, dropout_rate=0.2)

model = Model(inputs=input_layer, outputs=output_layer)

model.compile(loss='binary_crossentropy', 
            optimizer='Adam',
            metrics=['accuracy'])          

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 400)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 400, 300)     9000000     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 400, 300)     9000000     input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 400, 300)     0           embedding_1[0][0]                
__________________________________________________________________________________________________
spatial_dr

In [0]:
history = train_with_cv(model, batchSize=32, rocEvery = 2)

Train on 102124 samples, validate on 25532 samples
Epoch 1/32
 21024/102124 [=====>........................] - ETA: 1:21:31 - loss: 0.1189 - acc: 0.9606

102112/102124 [============================>.] - ETA: 0s - loss: 0.0742 - acc: 0.9745

102124/102124 [==============================] - 6566s 64ms/step - loss: 0.0742 - acc: 0.9745 - val_loss: 0.0501 - val_acc: 0.9803
Epoch 2/32
 10016/102124 [=>............................] - ETA: 1:32:28 - loss: 0.0572 - acc: 0.9795

102112/102124 [============================>.] - ETA: 0s - loss: 0.0539 - acc: 0.9801

102124/102124 [==============================] - 6538s 64ms/step - loss: 0.0539 - acc: 0.9801 - val_loss: 0.0438 - val_acc: 0.9832
roc-auc: 0.986 - roc-auc_val: 0.9829                                                                                                    
Epoch 3/32
  8608/102124 [=>............................] - ETA: 1:34:09 - loss: 0.0501 - acc: 0.9817

...

102124/102124 [==============================] - 6578s 64ms/step - loss: 0.0504 - acc: 0.9812 - val_loss: 0.0430 - val_acc: 0.9832
Epoch 4/32
 10016/102124 [=>............................] - ETA: 1:33:09 - loss: 0.0501 - acc: 0.9813

102112/102124 [============================>.] - ETA: 0s - loss: 0.0477 - acc: 0.9818

102124/102124 [==============================] - 6564s 64ms/step - loss: 0.0477 - acc: 0.9818 - val_loss: 0.0405 - val_acc: 0.9843
roc-auc: 0.9906 - roc-auc_val: 0.9874                                                                                                    
Epoch 5/32
  8576/102124 [=>............................] - ETA: 1:34:32 - loss: 0.0468 - acc: 0.9821

102112/102124 [============================>.] - ETA: 0s - loss: 0.0465 - acc: 0.9823

102124/102124 [==============================] - 6684s 65ms/step - loss: 0.0465 - acc: 0.9823 - val_loss: 0.0413 - val_acc: 0.9839
Epoch 6/32
  9184/102124 [=>............................] - ETA: 1:36:04 - loss: 0.0436 - acc: 0.9829

In [0]:
plot_history(history)

In [0]:
print(evaluate_on_test(model))

31915/31915 [==============================] - 55s 2ms/step
([0.04415003761226586, 0.9841088184271228], 0.9844031395273372)
